#  Using FABRIC NVMe Devices

Your compute nodes can include fast NVMe storage devices. These devices are made available as FABRIC components and can be added to your nodes like any other component.

This example notebook will demonstrate how to reserve and use FABRIC NVMe storage. 


## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
bastion_public_addr = 'bastion-1.fabric-testbed.net'
bastion_private_ipv4_addr = '192.168.11.226'
bastion_private_ipv6_addr = '2600:2701:5000:a902::c'

bastion_username = '<your bastion username>'

bastion_key_filename = os.environ['HOME'] + "/.ssh/id_rsa_fabric"

ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()


In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
print(f"FABRIC Credential Manager   : {credmgr_host}")

orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"FABRIC Orchestrator         : {orchestrator_host}")

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, 
                             cm_host=credmgr_host ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Create a Node

The cell below creates a slice that contains a single node with a 1TB NVMe device. 


### Set the Slice Name and FABRIC Site

In [ ]:
from fabrictestbed.slice_editor import ComponentModelType

slice_name="MySliceNVME2"
site_name="MAX"
node_name='Node1'
username='centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 16
disk = 100

nvme_name='nvme1'
nvme_model_type = ComponentModelType.NVME_P4510

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities 
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site_name)

# Set capacities
cap = Capacities()
cap.set_fields(core=cores, ram=ram, disk=disk)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Add the PCI NVMe device
n1.add_component(model_type=nvme_model_type, name=nvme_name)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                            slice_graph=slice_graph, 
                                            ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

## Get the Slice

In [ ]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    


return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)

print()
print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

## Get the Node

Retrieve the node information and save the management IP address.


In [ ]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

node = experiment_topology.nodes[node_name]

management_ip = str(node.get_property(pname='management_ip'))
print("Node Name        : {}".format(node.name))
print("Management IP    : {}".format(management_ip))
print()

nvme1 = node.components[nvme_name]
print("NVMe Name        : {}".format(nvme1.name))
print("Details          : {}".format(nvme1.details))
print("Disk (G)         : {}".format(nvme1.get_property(pname='capacity_allocations').disk))
print("Units            : {}".format(nvme1.get_property(pname='capacity_allocations').unit))
print("Model            : {}".format(nvme1.model))
print("Type             : {}".format(nvme1.get_property(pname='type')))


## Setup SSH Connection for Commands

Setup <code>paramiko</code> to send commands to the node using <code>ssh</code>.

In [ ]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
management_ip = str(node.get_property(pname='management_ip'))
print("Node {0} IP {1}".format(node_name, management_ip))

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)

bastion=paramiko.SSHClient()
bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_key_filename)


bastion_transport = bastion.get_transport()
if validIPAddress(management_ip) == 'IPv4':
    src_addr = (bastion_private_ipv4_addr, 22)
elif validIPAddress(management_ip) == 'IPv6':
    src_addr = (bastion_private_ipv6_addr, 22)
else:
    print('Management IP Invalid: {}'.format(management_ip))

dest_addr = (management_ip, 22)
bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

client.connect(management_ip,username=username,pkey = key, sock=bastion_channel)


## Configure the NVMe PCI Device

NVMe storage is provided as bare PCI block devices and will likely need to be partitioned and formated before use.

Run the command <code>lspci</code> to see your NVMe PCI device. This is the raw NVMe PCI device that is not yet formated or mounted.

In [ ]:
stdin, stdout, stderr = client.exec_command('lspci | grep NVMe')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

Run <code>fdisk</code> to see the NVMe block device. This device is not yet formated. We will partition and format this device later.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo fdisk -l /dev/nvme*')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

Find the NVMe block device.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo fdisk -l /dev/nvme*')

print (str(stdout.read(),'utf-8').replace('\\n','\n'))

There are no partitions or files systems on this device. First, let's partition the disk.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 mklabel gpt')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We can check that it has been created using the command below.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 print')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Next, let's print the free space in the NVMe drive in megabytes.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted /dev/nvme0n1 print unit MB print free')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

We can see that the disk is 1GB. Next, let's create a partition that uses up this entire space.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo parted --align optimal /dev/nvme0n1 mkpart primary ext4 0% 1000205MB')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Let's examine the partitions one more time to see the partition we created.

In [ ]:
stdin, stdout, stderr = client.exec_command('lsblk /dev/nvme0n1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now, let's format the disk with a `ext4` file system.

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo mkfs.ext4 /dev/nvme0n1p1')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Next, we need to mount the file system. 

In [ ]:
stdin, stdout, stderr = client.exec_command('sudo mkdir /mnt/nvme_mount && sudo mount /dev/nvme0n1p1 /mnt/nvme_mount')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Confirm that the file system is mounted and the correct size.

In [ ]:
stdin, stdout, stderr = client.exec_command('df -h /mnt/nvme_mount')
print(stdout.read().decode("utf-8"))
print(stderr.read().decode("utf-8"))

Now we can use the file system mounted at `/mnt/nvme_mount`


## Cleanup Your Experiment

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))